In [2]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.document_loaders import TextLoader,PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import VectorDBQA,RetrievalQA


_ = load_dotenv(find_dotenv())
openai.api_key = os.getenv('OPENAI_API_KEY')

llm = ChatOpenAI(model='gpt-4')




In [ ]:
### Load the PDF

In [12]:
loader = PyMuPDFLoader('the_nestle_hr_policy_pdf_2012.pdf')
documents=loader.load()

In [ ]:
###Create vector representations for text chunks

In [13]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=0)
chunks=text_splitter.split_documents(documents)
embeddings=OpenAIEmbeddings()
vectorstore=FAISS.from_documents(chunks,embeddings)

In [ ]:
### Build a question-answering system using the GPT-3.5 Turbo model to retrieve answers from text chunks.

In [14]:
qa=RetrievalQA.from_chain_type(llm = ChatOpenAI(model='gpt-3.5-turbo'),chain_type='stuff', retriever=vectorstore.as_retriever())

In [ ]:
### Create a prompt template to guide the chatbot in understanding and responding to users

In [15]:
template=""" You are a HR helpful assistant. Read the answer the following questions in english.
                Question:{question}
                Answer:
                """
prompt=PromptTemplate(input_variables=['question'],template=template)


In [23]:
question = "How is the working environment in Nestle"
results = vectorstore.similarity_search(question)

In [24]:
context = "\n\n".join([doc.page_content for doc in results])
context 

'Nestlé on the Move\n\t\nA flexible and dynamic organisation\n\nThe Nestlé Human Resources Policy\n5\nSince its founding, Nestlé has built a culture \nbased on values of trust, mutual respect and \ndialogue. Nestlé management and employees all \nover the world work daily to create and maintain \npositive individual and collective relationships, \nand are expected to do so as a core part of their \njob.\nNestlé not only upholds the freedom of \nassociation of its employees and the effective \nrecognition of the right to collective bargaining, \nbut also ensures that direct and frequent \ncommunication is established in the workplace. \nWhile dialogue with trade unions is essential, it \ndoes not replace the close relationship that our \nmanagement maintains with all employees.\nIn the spirit of continuous improvement, we \nencourage two-way dialogue with our employees \nthat goes beyond the traditional aspects of \ncollective bargaining in order to share knowledge \nand to jointly find 

In [25]:
question = "What is the reward for exceeding preformance "
results = vectorstore.similarity_search(question)
context = "\n\n".join([doc.page_content for doc in results])
context 

'further enables managers to acknowledge high \nperformance and reward employees accordingly, \nwhile ensuring low performance is properly \nmanaged with integrity.\nEmployees receive regular feedback on their \nperformance and career aspirations through \na variety of tools and processes such as the \nPerformance Evaluation process (PE), the  \nProgress and Development Guide (PDG) and  \n360° assessments. Each manager dedicates the \nnecessary time to the monitoring of objectives and \nregular coaching of employees through the year. \nEach employee, supported by the line manager, \nis in charge of her or his own professional \ndevelopment, whereby the employee is \nencouraged to express career objectives and \nexpectations in an open dialogue. We aim to retain \nand motivate employees by offering attractive but \nrealistic career moves allowing them to develop \ntheir skills in the long-term.\nGiven the importance Nestlé puts on cultural \ndiversity, employees who are interested in in

In [26]:
question = "What is the leave policy "
results = vectorstore.similarity_search(question)
context = "\n\n".join([doc.page_content for doc in results])
context 

'Corporate policy: \nPolicy on Conditions of Work and Employment\n\t\nEmployment and working conditions\n\nPolicy\nMandatory\nSeptember\u2009\u200a2012\nThe Nestlé  \nHuman Resources Policy\n\nnecessary for the continued development of \npeople and the Company.\nCorporate policy: \nExpatriation Policy\n\t\nTalent, development  \n\t\nand performance management\n\nThe Nestlé Human Resources Policy\n5\nSince its founding, Nestlé has built a culture \nbased on values of trust, mutual respect and \ndialogue. Nestlé management and employees all \nover the world work daily to create and maintain \npositive individual and collective relationships, \nand are expected to do so as a core part of their \njob.\nNestlé not only upholds the freedom of \nassociation of its employees and the effective \nrecognition of the right to collective bargaining, \nbut also ensures that direct and frequent \ncommunication is established in the workplace. \nWhile dialogue with trade unions is essential, it \ndoes

In [29]:
memory = ConversationBufferMemory(
    memory_key = 'chat_history',
    input_key='question')
prompt = PromptTemplate.from_template("""

Chat History : {chat_history}

Context : {context}

Question : {question}

Answer : 

""")

/tmp/ipykernel_95/440179681.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [31]:
retriever = vectorstore.as_retriever()

In [32]:
rag_chain = RetrievalQA.from_chain_type(
    llm = llm,
    retriever = retriever,
    chain_type_kwargs={
        "prompt" : prompt,
        "memory" : memory
    }
)

In [33]:
response = rag_chain.invoke("What are the working conditions")
response

{'query': 'What are the working conditions',
 'result': 'Nestlé emphasizes on safety and health within working areas, along with ensuring a respectful and trusting relationship among all employees. They do not tolerate any form of harassment or discrimination. The company also insists on adequate working conditions for all people working under contractual obligations with service providers. Additionally, Nestlé strives to attract and retain employees by focusing on Total Rewards, which include fixed pay, variable pay, benefits, personal growth, development, and a balanced work-life environment. The company also encourages its employees to have outside interests, especially community involvement.'}

In [34]:
response = rag_chain.invoke("How can i work to make those much better")
response

{'query': 'How can i work to make those much better',
 'result': "To improve the working conditions at a company like Nestlé, one could focus on a variety of strategies:\n\n1. Constant Feedback: Encourage a culture of open and honest communication. Implement systems to regularly check in with employees about their working conditions and satisfaction, and respond proactively to their suggestions and concerns.\n\n2. Enhance Employee Professional Development: Put an emphasis on training and development programs to help employees grow professionally. This can be achieved by investing in relevant training workshops and courses, fostering an environment of constant learning, and encouraging employees to pursue certifications or advanced degrees in their fields.\n\n3. Work-Life Balance: Prioritize work-life balance by ensuring employees aren't overworked, enabling flexible work schedules where possible, and encouraging employees to take time off for their well-being.\n\n4. Encourage Diversity